In [1]:
## Imports
import rheol_functions as rh
import numpy as np

## Parameters : 

# file_url = "E:\Data\Dissolution\Rheology\\" + "Carbopol_20220203_0p2pc.csv"
file_url = "D:\\Data\\Rheol\\AntonPaar\\" + "Carbopol ETD 2050 0p2pc - MCR502 - Rheol - Relax Strain Sweep a la Malvern - Relax Time Sweep - SandpaperCP50.csv"
# file_url = "D:\\Data\\Rheol\\Kinexus\\" + 'Carbopol_20220822_0p2pc_ETD_Relaxation_StrainSweep_LongExps.csv'

data = rh.read_rheology(file_url)

# # ## List steps, reassign steps 10 and 11 if they are ugly
list_of_steps = rh.list_steps(data)

# # # # ## Plot a flow curve, get the Herschel-Bulkley fits back
flow_curves = rh.slice(data, 'flowcurve')
HB_fits, fig_flowcurves = rh.plot_flowcurve(flow_curves, fit_from=1e-2, fit_up_to=1e3)

# # # ## Plot one or many amplitude sweeps
a_sweeps = rh.slice(data, 'asweep')
fig_asweep = rh.plot_asweep(a_sweeps)

# # # ## Plot a frequency sweep, selecting it from its step n°
f_sweeps = rh.slice(data, 10)    # should return 14
fig_fsweep = rh.plot_fsweep(f_sweeps)

# # # # ## Plot a time sweep 
# t_sweep = rh.slice(data, 'timesweep') # You can do it with step numbers even if it is wrong
# fig_tsweep = rh.plot_tsweep(t_sweep, x_axis_type='log', y_axis_type='log')

# # # ## Plot Normal forces for the flow curves
# # # fig_nforce = rh.plot_normalforce(flow_curves)

# # ## Shear startup (preshears)
# preshear = rh.slice(data, [0,14,18,22,26])
# fig_rev = rh.plot_startup(preshear, malvern=False)
# # # # fig_control = rh.plot_control_startup(tests, malvern=True)

## Creep plots
# creep = rh.slice(data, [20,24,28])
# fig_cr = rh.plot_creep(creep)

# # # Stress Relaxation plots after a step strain
# relax = rh.slice(data, 'stepstrain')
# fig_relax = rh.plot_stepstrain(relax, y_axis_type='linear')
# # fig_relax_2 = rh.plot_stepstrain_normalised(relax)

read_rheology > D:\Data\Rheol\AntonPaar\Carbopol ETD 2050 0p2pc - MCR502 - Rheol - Relax Strain Sweep a la Malvern - Relax Time Sweep - SandpaperCP50.csv is an Anton Paar file
assign_steps > Step : 3 too short for auto assignment
assign_steps > Step : 7 too short for auto assignment
assign_steps > Step : 11 too short for auto assignment
assign_steps > Step : 15 too short for auto assignment
assign_steps > Step : 19 too short for auto assignment
assign_steps > Step : 23 too short for auto assignment
assign_steps > Step : 26 too short for auto assignment
assign_steps > Step : 30 too short for auto assignment
assign_steps > Step : 33 too short for auto assignment
------------- Step list in DataFrame --------------------
  * Step n°0 with name  is a Preshear / 600.0 s
  * Step n°1 with name  is a Preshear / 120.0 s
  * Step n°2 with name  is a Timesweep / 1000.0 s
  * Step n°3 with name  is a Unknown / 0.5009 s
  * Step n°4 with name  is a Stepstrain / 1030.0 s
  * Step n°5 with name  is a

### Preshears 

In [11]:
# Preshears
from bokeh.plotting import figure, show, row 
from bokeh.models import Legend
from bokeh.palettes import Spectral7
from itertools import cycle

steps = range(1,25,4)
col = cycle(Spectral7)
fps_1 = figure(width=500, height=400, title='Preshear : ', y_range=(10,1001), y_axis_type='log')
fps_2 = figure(width=500, height=400, y_range=(1,1000), y_axis_type='log')
fevol = figure(width=500, height=400, title='Preshear : ', x_axis_type='log', y_range=(0,80), x_range=(10,3e5))

t0s, tends = np.zeros(np.size(steps)), np.zeros(np.size(steps))
s0s, sends = np.zeros(np.size(steps)), np.zeros(np.size(steps))

for no, step in enumerate(steps):
    dat_now = data[data['step'] == step]
    c = next(col)
    iend = dat_now['stress'].last_valid_index()

    if iend is not None:
        t0, tend = dat_now['time_global'].iloc[0], dat_now.loc[iend, 'time_global']
        s0, send = dat_now[dat_now['time'] > 1].iloc[0]['stress'], dat_now.loc[iend, 'stress']

        p1 = fps_1.scatter(dat_now['time'], dat_now['stress'], fill_color=c, line_color='black')
        p2 = fps_2.scatter(dat_now['time'], dat_now['shearrate'], fill_color=c, line_color='black', legend_label= 't = {:3.2f}'.format(t0) + ' s')

        s0s[no], sends[no] = s0, send
        t0s[no], tends[no] = t0, tend

fps_1.xaxis.axis_label, fps_2.xaxis.axis_label = 'time (s)', 'time (s)'
fps_1.yaxis.axis_label, fps_2.yaxis.axis_label = 'σ (Pa)', 'γ (1)'
fps_2.legend.location= 'top_right'

show(row(fps_1, fps_2))

fevol.scatter(t0s, s0s, marker='square', legend_label='Preshear : Start')
fevol.scatter(tends, sends, marker='circle', fill_color='white', legend_label='Preshear  : End')
fevol.xaxis.axis_label = 't (s)'
fevol.yaxis.axis_label = 'sigma (Pa)'
fevol.legend.location = 'bottom_left'
show(fevol)

### Time Sweeps 

In [12]:
# Time Sweeps

from bokeh.plotting import figure, show, row 
from bokeh.palettes import Spectral7
from itertools import cycle

steps = range(2,26,4)

col = cycle(Spectral7)
fwt = figure(width=500, height=400, title='Time Sweep (SAOS) : ', x_axis_type='log')
fevol = figure(width=500, height=400, title='Time Sweep (SAOS) : ', x_axis_type='log', y_range=(0,50), x_range=(10,3e5))
t0s, tends = np.zeros(np.size(steps)), np.zeros(np.size(steps))
s0s, sends = np.zeros(np.size(steps)), np.zeros(np.size(steps))

for no, step in enumerate(steps):
    dat_now = data[data['step'] == step]
    c = next(col)

    i0, iend = dat_now[dat_now['time'] > 10].first_valid_index(),  dat_now['stress'].last_valid_index()   
    if i0 is not None and iend is not None:
        t0, tend = dat_now.loc[i0, 'time_global'], dat_now.loc[iend, 'time_global']
        s0, send = dat_now.loc[i0, 'gprime'], dat_now.loc[iend, 'gprime']
        fwt.scatter(dat_now['time'], dat_now['gprime'], fill_color=c, line_color=rh.darken(c), legend_label = 't = {:3.2f}'.format(t0) + ' s')
        fwt.scatter(dat_now['time'], dat_now['gsecond'], fill_color=c, line_color=rh.darken(c), marker='square')

        s0s[no], sends[no] = s0, send
        t0s[no], tends[no] = t0, tend

fwt.xaxis.axis_label = 'time (s)'
fwt.yaxis.axis_label = 'G'', G" (Pa)'
fwt.legend.location = 'right'
show(fwt)

fevol.scatter(t0s, s0s, marker='square', legend_label='Wait LO : Start')
fevol.scatter(tends, sends, marker='circle', legend_label='Wait LO : End', fill_color='white')
fevol.xaxis.axis_label = 't (s)'
fevol.yaxis.axis_label = 'G'' (Pa)'
fevol.legend.location = 'bottom_left'
show(fevol)

## Strain Steps _à la Malvern_ 

With one short shear startup then one 'step strain' step with 0% strain

In [ ]:
# Strain steps 

from bokeh.plotting import figure, show, row, column
from bokeh.palettes import Spectral8
from itertools import cycle

# # Reverse PS ?
# steps_ps = range(22,50,4)
# steps_ramp = range(24,50,4) # MALVERN
# steps_relax = range(25,50,4)

# # # Squeeze flow ?
# # steps_ps = range(0,21,3)
# # steps_ramp = range(2,21,3) # MALVERN
# # steps_relax = range(3,21,3)

# # Squeeze flow ?
# steps_ps = range(51,79,4)
# steps_ramp = range(53,79,4) # MALVERN
# steps_relax = range(54,79,4)

# Squeeze flow ?
steps_ps = range(1,29,4)
steps_ramp = range(3,29,4) # MALVERN
steps_relax = range(4,29,4)

col = cycle(Spectral8)
fstrn_left = figure(width=250, height=400, title='Ramp', y_range=(-0,800), x_range=(-10,0), toolbar_location=None)
fstrn_right = figure(width=800, height=400, title='Relax', y_range=(-0,800), x_axis_type='log', x_range=(0.01,2e4), toolbar_location='above', y_axis_location=None)
fstrs_left = figure(width=250, height=400, title='', y_range=(0.1,100.01), x_range=(-10,0), toolbar_location=None, y_axis_type='log')
fstrs_right = figure(width=800, height=400, title='', y_range=(0.1,100.01), x_axis_type='log', x_range=(0.01,2e4), toolbar_location='below', y_axis_location=None, y_axis_type='log')

for (sps, sramp, srel) in zip(steps_ps, steps_ramp, steps_relax):
    ramp_now = data[data['step'] == sramp]
    wait_now = data[data['step'] == sps + 1]
    rel_now = data[data['step'] == srel]
    ps_now = data[data['step'] == sps]

    wait = wait_now['time'].iloc[-1]
    c, t_ramp, stn_ramp = next(col), ramp_now['time'].iloc[-1], ramp_now['strain'].iloc[-1]
    strs0 = rel_now['stress'].iloc[0]
    gammadot = -0.01*stn_ramp/t_ramp

    fstrn_left.scatter(ramp_now['time'] - t_ramp, ramp_now['strain'], fill_color=c, line_color=rh.darken(c))
    fstrn_right.scatter(rel_now['time'], rel_now['strain'] + stn_ramp, fill_color=c, line_color=rh.darken(c))
    fstrs_left.scatter(ramp_now['time'] - t_ramp, ramp_now['stress'], fill_color=c, line_color=rh.darken(c))
    fstrs_right.scatter(rel_now['time'], rel_now['stress'], fill_color=c, line_color=rh.darken(c), legend_label = 'gamma = {:3.1f}'.format(stn_ramp) + '')

fstrn_left.xaxis.axis_label, fstrn_left.yaxis.axis_label = '', 'γ (%)'
fstrs_left.xaxis.axis_label, fstrs_left.yaxis.axis_label = 't (s)', 'σ (Pa)'
fstrs_right.xaxis.axis_label = 't (s)'
fstrs_right.legend.location = 'bottom_left'

# frel.xaxis.axis_label, frel.yaxis.axis_label = 'time (s)', ''
# fstrn.legend.location = 'right'

show(column(row(fstrn_left, fstrn_right),row(fstrs_left, fstrs_right)))

## Regular Step strains for TA and Anton Paar

With only one step strain

In [ ]:
# CHECK

from bokeh.plotting import figure, show, row, column
from bokeh.palettes import Spectral4
from itertools import cycle

steps_strain = [3,6,9,12,15]
steps_wait = [n-1 for n in steps_strain]
steps_ps = [n-2 for n in steps_strain]

col = cycle(Spectral4)
fstrn = figure(width=800, height=400, title='Relax : Strain', x_axis_type='log', x_axis_location=None, toolbar_location='above')
fstrs = figure(width=800, height=400, title='Relax : Stress',  x_axis_type='log', toolbar_location='right', y_axis_type='log')

for (sps, swt, stepstn) in zip(steps_ps, steps_wait, steps_strain):

    ps_now = data[data['step'] == sps]
    swt_now = data[data['step'] == swt]
    stepstn_now = data[data['step'] == stepstn]

    c = next(col)
    dc = rh.darken(c)
    tw, gamma, strs0 = swt_now['time'].iloc[-1], np.max(stepstn_now['strain']), stepstn_now['stress'].iloc[1]

    fstrn.scatter(stepstn_now['time'], stepstn_now['strain'], fill_color=c, line_color=dc)
    fstrs.scatter(stepstn_now['time'], stepstn_now['stress'], fill_color=c, line_color=dc, legend_label=' gamma = {:3.1f}'.format(gamma) + ' s')
    

fstrn.xaxis.axis_label, fstrn.yaxis.axis_label = '', 'γ (%)'
fstrs.xaxis.axis_label, fstrs.yaxis.axis_label = 't (s)', 'σ (Pa)'

show(column(fstrn, fstrs))

In [ ]:
from bokeh.plotting import show
fig_flowcurves.y_range.start, fig_flowcurves.y_range.end = 1, 100
fig_asweep.y_range.start, fig_asweep.y_range.end = 1, 100
fig_fsweep.y_range.start, fig_fsweep.y_range.end = 1, 100
show(fig_flowcurves)
show(fig_asweep)
show(fig_fsweep)

In [ ]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Patch, LinearColorMapper, Arrow, VeeHead, ColorBar
from bokeh.palettes import Magma256
from itertools import cycle

theta = np.linspace(-np.pi, np.pi, 360)
xy = lambda r, theta : (r*np.cos(theta), r*np.sin(theta))
eps = lambda r, R0, R : (1 + (R0**3 - R**3)/r**3)**(2/3) - 1
e_max = eps(1.2,1,1.2)
c_idx = lambda val : round(255*val/e_max)
color_map = Magma256
cmapper = LinearColorMapper(low=0, high=-e_max, palette=color_map)
color_bar = ColorBar(color_mapper=cmapper)

f = figure(x_range=(-2.2,2.2), y_range=(-2.2,2.2), width=500, height=400, match_aspect=True)
source_bg = ColumnDataSource(data={'x':[-3,3,3,-3], 'y':[3,3,-3,-3]})
glyph_bg = Patch(x='x', y='y', fill_color='black')
f.add_glyph(source_bg, glyph_bg)

r_list = np.linspace(3,1.2,64)
for r in r_list:
    x,y = xy(r, theta)
    c_now = color_map[c_idx(eps(r, 1, 1.2))]
    source = ColumnDataSource(data={'x':x, 'y':y})
    glyph = Patch(x='x', y='y', fill_color=c_now, line_color=c_now)
    f.add_glyph(source, glyph)

xb,yb = xy(1.2, theta)
glyph_bb = Patch(x='x', y='y', fill_color='white', line_color='black')
source_bb = ColumnDataSource(data={'x':xb, 'y':yb})
f.add_glyph(source_bb, glyph_bb)

x0,y0 = xy(1.0, theta)
f.line(x0, y0, line_dash='dashed', line_color='gray')

# f.image([[[-e_max, 0],[0, -e_max]]], dw=0.5, dh=0.5, x=0, y=0, color_mapper=cmapper)

for th in np.linspace(-np.pi, np.pi*(1-1/10), 20):

    f.add_layout(Arrow(end=VeeHead(size=5), line_color="black",
        x_start=1.02*np.cos(th), y_start=1.02*np.sin(th), 
        x_end=1.18*np.cos(th), y_end=1.18*np.sin(th)))

    # f.line([1.2*np.cos(th), 3*np.cos(th)], [1.2*np.sin(th), 3*np.sin(th)], line_color='lightgray', line_dash='dotted')

f.add_layout(color_bar, place='right')
f.xaxis.axis_label, f.yaxis.axis_label ='x', 'y'

show(f)

In [ ]:
4/3*np.pi*5**-3

0.033510321638291124